# Zone Temperature Dynamical Model

- investigate different physics-informed dynamical models for zone temperature prediction

In [1]:
from jax import config 
config.update("jax_debug_nans", False)
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

import jax 
import jax.numpy as jnp 
import flax.linen as nn
import optax 
import pandas as pd 

from dynax.core.base_block_state_space import BaseContinuousBlockSSM
from dynax.simulators.simulator import DifferentiableSimulator
from dynax.trainer.train_state import TrainState

## 0. Data
- EP data with prototypical building model
- Real data from Lennox thermostat data

In [ ]:
"""
import pandas as pd 

dates = ['01-'+str(i)+'-18' for i in range(14, 28)]
data = pd.read_excel('./data/real-world-data.xls', sheet_name=dates, header=3)

combined_data = pd.DataFrame()
for key in data.keys():
    combined_data = pd.concat([combined_data, data[key]], axis=0, ignore_index=True)

combined_data.to_csv('./data/real-world-data.csv')
"""

In [ ]:
data = pd.read_csv('./data/real-world-data.csv')

columns_mapper = {'LocalTime':'time', 'tabtsense0':'temp_zone', 'SP_cool':'csp', 'SP_heat':'hsp', 'CoolDemand':'qcool', 'HeatDemand':'qheat', 'humidityindoor_0':'rh_zone', 'oupresentcompressorfreq':'freq', 'iuairflowrate':'cfm', 'surf_tempf':'temp_out', 'rad':'qsol'}

data = data[columns_mapper.keys()].rename(columns = columns_mapper)
data.index = pd.DatetimeIndex(data['time'], freq='1min')
data = data.drop(columns=['time'])


In [ ]:
data.head()

This data is from Rohini's house, which has a Lennox thermostat but Carrier unit. Thus the equipment info are basically not available. 
- assume nominal heating capacity is 4 ton (e.g., 14 kW)

In [ ]:
# change IP unit to SI
# F to C
data[['temp_zone', 'csp', 'hsp', 'temp_out']] =  5./9*(data[['temp_zone', 'csp', 'hsp', 'temp_out']] - 32)
# W to kW
data['qsol'] /= 1000.
# percentage to kW
data[['qcool', 'qheat']] *= 14./100.

In [ ]:
# resampel to 15min 
data = data.resample('15min').mean()
data.head()

In [ ]:
# add lag to target
def add_lags(data, cols, lags):
    for lag in range(1, lags+1):
        for col in cols:
            data[col+'-'+str(lag)] = data[col].shift(lag)
    return data



## 1. Models

Try two different mdoels
- lumped single zone dynamic model 
- neural ode


### 1.2 Neural ODE

For realistic settings, here I only assume known measurements, such as outdoor air temperature, solar radiation, and energy usage

$$
    \frac{dT_z}{dt} = f(h_t, T_o, \dot q_{sol}, \dot q_{hvac}; \theta)
$$

#### Prepare Data

In [ ]:
# construct time feature
data['hour'] = data.index.hour
data['weekday'] = data.index.weekday


In [ ]:
data

In [ ]:
target_col = 'temp_zone'
feature_col = ['qheat', 'temp_out', 'qsol', 'hour', 'weekday']

lags = 8
cols = [target_col] + feature_col
data2 = add_lags(data[cols], cols, lags)
data2 = data2.dropna()
print(data2.head())

# split training and testing
ratio = 0.5
n_train = int(len(data2)*ratio)
data_train = data2.iloc[:n_train, :]
data_test = data2.iloc[n_train:, :]

# get cols with lags
lag_cols = []
if lags:
    for lag in range(1, lags):
        for col in feature_col:
            lag_cols.append(col + '-' + str(lag))

# to model signals
x_train = jnp.array(data_train.loc[:, 'temp_zone-1'].values) 
u_train = jnp.array(data_train.loc[:, lag_cols + feature_col].values)
y_train = jnp.array(data_train.loc[:, target_col].values)

# normalize
u_mean = u_train.mean(axis=0)
u_std = u_train.std(axis=0)
u_train = (u_train - u_mean) / u_std

# reshape for model
x_train = jnp.reshape(x_train, (-1,1))
u_train = jnp.expand_dims(u_train, axis=1)
y_train = jnp.reshape(y_train, (-1,1)) 

print("we have a training data set of :", u_train.shape[0])


x_test = jnp.array(data_test.loc[:, 'temp_zone-1'].values) 
u_test = jnp.array(data_test.loc[:, lag_cols + feature_col].values)
y_test = jnp.array(data_test.loc[:, target_col].values)
u_test = (u_test - u_mean) / u_std
x_test = jnp.reshape(x_test, (-1,1))
u_test = jnp.expand_dims(u_test, axis=1)
y_test = jnp.reshape(y_test, (-1,1))
print("we have a training data set of :", u_test.shape[0])



#### Define Model

In [ ]:
INPUT_DIM = u_train.shape[-1]

class NeuralODE(BaseContinuousBlockSSM):
    state_dim: int = 1
    input_dim: int = INPUT_DIM
    output_dim: int = 1

    def setup(self):
        super().setup()
        self._fx = self.fx(output_dim = self.state_dim)
        self._fy = self.fy(output_dim = self.output_dim)
    
    def __call__(self, states, inputs):
        # states: (B, nx)
        # inputs: (B, ni)
        assert self.input_dim == jnp.shape(inputs)[-1]
        return super().__call__(states, inputs)

    class fx(nn.Module):
        output_dim: int 
        def setup(self):
            #self.dense1 = nn.Dense(features=4, use_bias=False)
            self.dense = nn.Dense(features=self.output_dim, use_bias=False)
        
        def __call__(self, states, inputs):
            out = self.dense(inputs)
            #out = nn.relu(out)
            #out = self.dense(out)
            return out

    
    class fy(nn.Module):
        output_dim: int 

        def __call__(self, states, inputs):
            # a faked value
            return states @ jnp.eye(1)

ode = NeuralODE()
key = jax.random.PRNGKey(2023)
init_params = ode.init(key, jnp.ones((4,1)), jnp.ones((4,INPUT_DIM)))
rhs, out = ode.apply(init_params, jnp.ones((4,1)), jnp.ones((4,INPUT_DIM)))
print(ode.tabulate(key, jnp.ones((4,1)), jnp.ones((4,INPUT_DIM))))

In [ ]:
class NeuralModel(nn.Module):
    
    state_dim: int = 1
    ts: int = 0
    dt: int = 900

    @nn.compact 
    def __call__(self, states, inputs):

        dynamics = NeuralODE(state_dim = self.state_dim)
        states_out, measures_out = DifferentiableSimulator(
            dynamics, 
            dt=self.dt, 
            mode_interp='linear', 
            start_time=self.ts
        )(states, inputs)
        
        return jnp.squeeze(states_out, axis=-1)

model = NeuralModel(state_dim=1)

key = jax.random.PRNGKey(2023)
init_params = model.init(key, jnp.ones((1)), jnp.ones((4,INPUT_DIM)))
init_out = model.apply(init_params, jnp.ones((1,)), jnp.ones((4,INPUT_DIM)))
print(model.tabulate(key, jnp.ones((1,)), jnp.ones((4,INPUT_DIM))))

In [ ]:
# batch Model with shared parameters
VModel = nn.vmap(NeuralModel, in_axes=(0,0), out_axes=0, variable_axes={'params': None}, split_rngs={'params': False})
vmodel = VModel(state_dim=1)
init_params = vmodel.init(key, jnp.ones((32,1)), jnp.ones((32,4,INPUT_DIM)))
out = vmodel.apply(init_params, jnp.ones((32,1)), jnp.ones((32,4,INPUT_DIM)))
print(out.shape)

#### Training and Evaluation

In [ ]:
# inverse simulation train_step
@jax.jit
def train_step(train_state, state_init, u, target):
    def mse_loss(params):
        # prediction
        outputs_pred = train_state.apply_fn(params, state_init, u)
        #outputs_pred = jnp.clip(outputs_pred, 16., 40.)
        # mse loss: match dimensions
        pred_loss = jnp.mean((outputs_pred - target)**2)
        return pred_loss
    
    loss, grad = jax.value_and_grad(mse_loss)(train_state.params)
    train_state = train_state.apply_gradients(grads=grad)

    return loss, grad, train_state

schedule = optax.linear_schedule(
    init_value = 1e-1, 
    transition_steps = 5000, 
    transition_begin=0, 
    end_value=1e-4
)

optim = optax.chain(
    #optax.clip_by_global_norm(1.0),
    #optax.clip(1.),
    #optax.scale(1.2),
    #optax.lamb(1e-04),
    optax.adamw(1e-3)
)


In [ ]:
train_state = TrainState.create(
    apply_fn=vmodel.apply,
    params=init_params,
    tx = optim,
)


In [ ]:
x_train.shape, u_train.shape 

In [ ]:
n_epochs = 150000

# train loop
for epoch in range(n_epochs):
    loss, grad, train_state = train_step(train_state, x_train, u_train, y_train)
    if epoch % 1000 == 0:
        print(f"epoch: {epoch}, loss: {loss}")

In [ ]:
y_pred = vmodel.apply(train_state.params, x_train, u_train)
y_pred_test = vmodel.apply(train_state.params, x_test, u_test)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_train[:,:], label='target')
plt.plot(y_pred[:,:], label='pred')
plt.legend()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test[:,:], label='target')
plt.plot(y_pred_test[:,:], label='pred')
plt.legend()

### 1.3 Physics-informed Neural ODE

In this section, I will use physics-informed neural ODE to learn the dynamics of the system.

$$

    mC_{p,a}\frac{dT_z}{dt} = q_{hvac} + q_{int} + q_{intz} + q_{inf} + q_{surf}
$$

or 

$$
    \frac{dT_z}{dt} = \frac{1}{C}(q_{hvac} + q_{inf} + q_{int} + q_{intz} + q_{surf})  
$$

$$
    \frac{dT_z}{dt} = a \dot q_{hvac} + b (T_{out} - T_z) + c(T_{z,i} - T_z) + \frac{1}{C} (\dot q_{int} + \dot q_{surf}) =  a \dot q_{hvac} + b (T_{out} - T_z) + c(T_{z,i} - T_z) + D_{ann}(t, \dot q_{sol}, T_{out}) = -(b+c)T_z + a\dot q_{hvac} + bT_{out} + cT_{z,i} + D_{ann}(t, \dot q_{sol}, T_{out})

$$

Here we wll model this in dynax as a differentiable equation.

To simplify for residential buildings, where there is typically one zone, we can set $c=0$.


**Thoughts to try**:
- the $a, b, c$ might be too small, here I use the inverse
- the influence of solar/internal heat gain should not depend on zone temperature. Therefore, the nonlinear disturbance has no dependency on states.
- try out physics-informed deep koopman modeling


#### Prepare Data

`states`: $T_z$

`inputs`: $(\dot q_{hvac}|_{t}^{t+H}, (t, \dot q_{sol}, T_{out})|_{t-P}^{t+H})$

In [ ]:
target_col = 'temp_zone'
feature_col = ['qheat', 'temp_out', 'qsol', 'hour', 'weekday']

lags = 8
cols = [target_col] + feature_col
data3 = add_lags(data[cols], cols, lags)
data3 = data3.dropna()
print(data3.head())

# split training and testing
ratio = 0.25
n_train = int(len(data3)*ratio)
data_train = data3.iloc[:n_train, :]
data_test = data3.iloc[n_train:, :]

# get cols with lags
lag_cols = []
if lags:
    for lag in range(1, lags):
        for col in feature_col:
            lag_cols.append(col + '-' + str(lag))

# to model signals
x_train = jnp.array(data_train.loc[:, 'temp_zone-1'].values) 
u_train = jnp.array(data_train.loc[:, feature_col + lag_cols].values)
y_train = jnp.array(data_train.loc[:, target_col].values)

# normalize
normalize=False
if normalize:
    u_mean = u_train.mean(axis=0)
    u_std = u_train.std(axis=0)
    u_train = (u_train - u_mean) / u_std

# reshape for model
x_train = jnp.reshape(x_train, (-1,1))
u_train = jnp.expand_dims(u_train, axis=1)
y_train = jnp.reshape(y_train, (-1,1)) 

print("we have a training data set of :", u_train.shape[0])


x_test = jnp.array(data_test.loc[:, 'temp_zone-1'].values) 
u_test = jnp.array(data_test.loc[:, feature_col + lag_cols].values)
y_test = jnp.array(data_test.loc[:, target_col].values)
if normalize:
    u_test = (u_test - u_mean) / u_std
x_test = jnp.reshape(x_test, (-1,1))
u_test = jnp.expand_dims(u_test, axis=1)
y_test = jnp.reshape(y_test, (-1,1))
print("we have a training data set of :", u_test.shape[0])


#### Define Model

In [2]:
# An encoder-decoder model to predict extegonous disturbances on the dynamic system such as solar gain and internal gains. 
class EncoderDecoder(nn.Module):
    hidden_dim: int = 32
    
    @nn.compact
    def __call__(self, encoder_inputs, decoder_inputs):
        # encoder_inputs: (B, T, Ni)
        # decoder_inputs: (B, T, No)
        # encoder 
        encoder = nn.RNN(
            nn.LSTMCell(features=self.hidden_dim),
            return_carry=True,
            name = "encoder"
        )
        encoder_state, _ = encoder(encoder_inputs)
        # decoder 
        decoder = nn.RNN(
            nn.LSTMCell(features=self.hidden_dim),
            return_carry=False,
            name = "decoder"
        )
        decoder_outputs = decoder(decoder_inputs, initial_carry=encoder_state)

        return decoder_outputs

encoder_decoder = EncoderDecoder(hidden_dim=16)
key = jax.random.PRNGKey(2023)
init_params = encoder_decoder.init(key, jnp.ones((4, 24, 8)), jnp.ones((4, 12, 8)))
out = encoder_decoder.apply(init_params, jnp.ones((4, 24, 8)), jnp.ones((4, 12, 8)))
out.shape

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


AttributeError: module 'flax.linen' has no attribute 'RNN'

In [5]:
import flax
flax.__version__

'0.6.4'

In [ ]:
INPUT_DIM = u_train.shape[-1]
class PINN(BaseContinuousBlockSSM):
    state_dim: int = 1
    input_dim: int = INPUT_DIM
    output_dim: int = 1

    def setup(self):
        super().setup()
        self._fx = self.fx(output_dim = self.state_dim)
        self._fy = self.fy(output_dim = self.output_dim)
    
    def __call__(self, states, inputs):
        # states: (B, nx)
        # inputs: (B, ni)
        return super().__call__(states, inputs)

    class fx(nn.Module):
        output_dim: int 
        def setup(self):
            self.B = self.param('B', nn.initializers.constant(1000.), ()) # 
            self.C = self.param('C', nn.initializers.constant(1000.), ()) # room capacitance

            self.disturbance = nn.Dense(features=self.output_dim, use_bias=False)
        
        def __call__(self, states, inputs):
            # assumed the first input is q_hvac
            q_hvac = jnp.take(inputs, jnp.array([0]), axis=-1)
            others = jnp.take(inputs, jnp.arange(1, jnp.shape(inputs)[-1]), axis=-1)
            # run model
            disturbance = self.disturbance(others)
            return 1./self.C * (q_hvac + disturbance)
    
    class fy(nn.Module):
        output_dim: int 

        def __call__(self, states, inputs):
            # a faked value
            return states @ jnp.eye(1)

ode = PINN()
key = jax.random.PRNGKey(2023)
init_params = ode.init(key, jnp.ones((4,1)), jnp.ones((4,32)))
rhs, out = ode.apply(init_params, jnp.ones((4,1)), jnp.ones((4,32)))
print(ode.tabulate(key, jnp.ones((4,1)), jnp.ones((4,32))))

In [ ]:
class PINNModel(nn.Module):
    
    state_dim: int = 1
    ts: int = 0
    dt: int = 900
    regularizer:callable = nn.relu

    @nn.compact 
    def __call__(self, states, inputs):

        dynamics = PINN(state_dim = self.state_dim, name='dynamic')
        states_out, measures_out = DifferentiableSimulator(
            dynamics, 
            dt=self.dt, 
            mode_interp='linear', 
            start_time=self.ts
        )(states, inputs)
        
        # add regularizer for loss calculation
        if self.regularizer is not None:
            # a reduce function to get the last value in sow(), which stores previous value
            reduce_fn = lambda a, b: b 
            param_dynamics = dynamics.variables['params']['_fx']['C']
            self.sow('reg_loss', 'C', self.regularizer(param_dynamics), reduce_fn=reduce_fn)

        return jnp.squeeze(states_out, axis=-1)

In [ ]:
# added a parameter regulizer
params_bounds = [500, 100000]
def param_regularizer(params, params_bounds):
    lb, ub = params_bounds
    loss = nn.relu(params - ub) + nn.relu(lb - params)
    return loss

In [ ]:
model = PINNModel(state_dim=1, regularizer=lambda params: param_regularizer(params, params_bounds))

key = jax.random.PRNGKey(2023)
init_params = model.init(key, jnp.ones((1)), jnp.ones((4,32)))
init_out, model_states = model.apply(init_params, jnp.ones((1,)), jnp.ones((4,32)), mutable=['reg_loss'])
print(model_states)
print(model.tabulate(key, jnp.ones((1,)), jnp.ones((4,32))))

In [ ]:
# batch Model with shared parameters
VModel = nn.vmap(PINNModel, in_axes=(0,0), out_axes=0, variable_axes={'params': None, 'reg_loss': None}, split_rngs={'params': False})
vmodel = VModel(state_dim=1, regularizer=lambda params: param_regularizer(params, params_bounds))
init_params = vmodel.init(key, jnp.ones((32,1)), jnp.ones((32,4,INPUT_DIM)))
out, states = vmodel.apply(init_params, jnp.ones((32,1)), jnp.ones((32,4,INPUT_DIM)), mutable=['reg_loss'])
print(out.shape)

#### Training and Evaluation

In [ ]:
u_train.shape

In [ ]:
qhvac = jnp.take(u_train, jnp.array([0]), axis=-1)
qhvac.shape

others = jnp.take(u_train, jnp.arange(1, INPUT_DIM), axis=-1)
others.shape

In [ ]:
# inverse simulation train_step
@jax.jit
def train_step(train_state, state_init, u, target):
    def mse_loss(params):
        # prediction
        outputs_pred, regulizers = train_state.apply_fn(params, state_init, u, mutable=['reg_loss'])
        #outputs_pred = jnp.clip(outputs_pred, 16., 40.)
        # mse loss: match dimensions
        pred_loss = jnp.mean((outputs_pred - target)**2)
        reg_loss = sum(jax.tree_util.tree_leaves(regulizers["reg_loss"]))

        return pred_loss + reg_loss
    
    loss, grad = jax.value_and_grad(mse_loss)(train_state.params)
    train_state = train_state.apply_gradients(grads=grad)

    return loss, grad, train_state

schedule = optax.linear_schedule(
    init_value = 1e-1, 
    transition_steps = 5000, 
    transition_begin=0, 
    end_value=1e-4
)

optim = optax.chain(
    #optax.clip_by_global_norm(1.0),
    #optax.clip(1.),
    #optax.scale(1.2),
    #optax.lamb(1e-04),
    optax.adam(0.1)
)

train_state = TrainState.create(
    apply_fn=vmodel.apply,
    params=init_params,
    tx = optim,
)


In [ ]:
optim = optax.chain(
    #optax.clip_by_global_norm(1.0),
    #optax.clip(1.),
    #optax.scale(1.2),
    #optax.lamb(1e-03),
    optax.adam(0.01)
)

In [ ]:
n_epochs = 500000

# train loop
for epoch in range(n_epochs):
    loss, grad, train_state = train_step(train_state, x_train, u_train, y_train)
    if epoch % 1000 == 0:
        print(f"epoch: {epoch}, loss: {loss}")

In [ ]:
loss

In [ ]:
grad

In [ ]:
train_state.params

In [ ]:
y_pred = vmodel.apply(train_state.params, x_train, u_train)
y_pred_test = vmodel.apply(train_state.params, x_test, u_test)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_train[:,:], label='target')
plt.plot(y_pred[:,:], label='pred')
plt.legend()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test[:,:], label='target')
plt.plot(y_pred_test[:,:], label='pred')
plt.xlabel('day', fontsize=14)
plt.ylabel('temperature [C]', fontsize=14)
plt.legend()

In [ ]:
# simple free-floating test
import numpy as np 
u_test1 = np.array(u_test)
u_test1[:, 0] = 0

y_pred_test1 = vmodel.apply(train_state.params, x_test, u_test1)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test[:,:], label='target')
plt.plot(y_pred_test[:,:], label='pred')
plt.plot(y_pred_test1[:,:], label='-')
plt.legend()

### 1.4 Koopman Model

## Physics-Consistent Check

How to know if the learned model follows general physics? 
- the derivative of room temp with respect to the HVAC energy rate should be positive. The more heat injected to the room, the higher the room temperature.

In [ ]:
u_fake = u_train[[100], ::]
x_fake = x_train[[100], ::]

In [ ]:
u_fake.shape, x_fake.shape

In [ ]:
#create 10 different points: 0-10 kW

q_hvac = jnp.array([i for i in range(-10, 10)])
n_points = len(q_hvac)

others = jnp.tile(u_fake[:,:,1:], reps=(n_points, 1, 1))
q_hvac = q_hvac.reshape(-1,1)[:, jnp.newaxis, :]
q_hvac.shape, others.shape

In [ ]:
u = jnp.concatenate((q_hvac, others), axis=-1)
x = jnp.tile(x_fake, reps=(n_points, 1))

In [ ]:
y_pred = vmodel.apply(train_state.params, x, u)

In [ ]:
plt.figure()
plt.plot(q_hvac.squeeze(), y_pred, label='predicted') #
plt.plot(q_hvac.squeeze(), x, label='previous temp') # previous temperature
plt.grid()